# Объединение (соединение) объектов DataFrame

In [1]:
import numpy as np
import pandas as pd

## Конкатенация

Напрямую "склеивает" объекты dataframe вместе.

In [2]:
data_one = {'A': ['A0', 'A1', 'A2', 'A3'],'B': ['B0', 'B1', 'B2', 'B3']}

In [3]:
data_two = {'C': ['C0', 'C1', 'C2', 'C3'], 'D': ['D0', 'D1', 'D2', 'D3']}

In [4]:
one = pd.DataFrame(data_one)

In [5]:
two = pd.DataFrame(data_two)

In [6]:
one

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [7]:
two

,C,D
0,C0,D0
1,C1,D1
2,C2,D2
3,C3,D3


## Axis = 0 

### Конкатенация по строкам

In [8]:
axis0 = pd.concat([one,two],axis=0)

In [9]:
axis0

,A,B,C,D
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
2,A2,B2,NaN,NaN
3,A3,B3,NaN,NaN
0,NaN,NaN,C0,D0
1,NaN,NaN,C1,D1
2,NaN,NaN,C2,D2
3,NaN,NaN,C3,D3


## Axis = 1

### Конкатенация по колонкам

In [10]:
axis1 = pd.concat([one,two],axis=1)

In [11]:
axis1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


### Axis 0 , но колонки совпадают
**В случае, если Вам нужно следующие:**

In [12]:
two.columns = one.columns

In [13]:
pd.concat([one,two])

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3
0,C0,D0
1,C1,D1
2,C2,D2
3,C3,D3


# Соединение (Merge)

## Таблицы с данными

In [14]:
registrations = pd.DataFrame({'reg_id':[1,2,3,4],'name':['Andrew','Bobo','Claire','David']})
logins = pd.DataFrame({'log_id':[1,2,3,4],'name':['Xavier','Andrew','Yolanda','Bobo']})

In [15]:
registrations

,reg_id,name
0,1,Andrew
1,2,Bobo
2,3,Claire
3,4,David


In [16]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


# pd.merge()

Соединяет объекты pandas DataFrame на основе ключевых колонок, по аналогии с соединением join в SQL. Различные виды соединения (внешнее outer, внутреннее inner) можно указать с помощью параметра **how**.

-----

# Виды соединений - Inner, Left, Right и Outer Joins

## Внутреннее соединение - Inner Join

**Соединяет строки, для которых ключ присутствует в ОБЕИХ таблицах. В результате такого соединения мы не получим значений NaN, потому что по определению значения в ключевых колонках должны существовать в обеих таблицах.**
**Если посмотреть на таблицы registrations и logins, то только два человека -  Andrew и Bobo - присутствуют в обеих таблицах.**

In [18]:
# Обратите внимание, что в отличие от concat, pd.merge не принимает на вход список
pd.merge(registrations,logins,how='inner',on='name')

,reg_id,name,log_id
0,1,Andrew,2
1,2,Bobo,4


In [19]:
# Pandas может сам определить ключевую колонку (параметр on) в случае, 
# если только одна из колонок совпадает в двух таблицах
pd.merge(registrations,logins,how='inner')

,reg_id,name,log_id
0,1,Andrew,2
1,2,Bobo,4


In [20]:
# Pandas возвращает ошибку, если колонка "on" не существует в какой-то из двух таблиц
# pd.merge(registrations,logins,how='inner',on='reg_id')

---

## Левое соединение - Left Join

**Находит соответствующие строки И включает в результат все строки из левой таблицы (если смотреть на список таблиц - левая таблица это та таблица, которая находится слева в этом списке).**

**Выводим все строки из левой таблицы registrations, и если у них нет строки в таблице logins, то заполняем колонки значениями NaN.**

In [21]:
pd.merge(registrations,logins,how='left')

,reg_id,name,log_id
0,1,Andrew,2.0
1,2,Bobo,4.0
2,3,Claire,NaN
3,4,David,NaN


## Правое соединение - Right Join
**Находит соответствующие строки И включает в результат все строки из правой таблицы.**

**Отобразить все строки из правой таблицы, и если для них нет соответствия в левой таблице, то записать в колонки из левой таблицы значения NaN.**

In [22]:
pd.merge(registrations,logins,how='right')

,reg_id,name,log_id
0,NaN,Xavier,1
1,1.0,Andrew,2
2,NaN,Yolanda,3
3,2.0,Bobo,4


## Внешнее соединение - Outer Join

**Показать строки, которые нашлись или в левой, или в правой таблице.**
**В нашем примере это все строки из таблицы registrations и из таблицы logins. Там, где не нашлись соответствующие строки, пишем значения NaN.**

**Замечание - если сравнивать это терминологию с языком SQL, то в SQL эта операция называется FULL OUTER JOIN. А операции LEFT JOIN и RIGHT JOIN в языке SQL называются соответственно LEFT OUTER JOIN и RIGHT OUTER JOIN).**

In [23]:
pd.merge(registrations,logins,how='outer')

,reg_id,name,log_id
0,1.0,Andrew,2.0
1,2.0,Bobo,4.0
2,3.0,Claire,NaN
3,4.0,David,NaN
4,NaN,Xavier,1.0
5,NaN,Yolanda,3.0


## Соединение по индексу или колонке

**Чтобы явно указать, по какому индексу или колонке выполнять соединение, можно использовать параметры left_on, right_on, left_index, right_index**

In [24]:
registrations

,reg_id,name
0,1,Andrew
1,2,Bobo
2,3,Claire
3,4,David


In [25]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [26]:
registrations = registrations.set_index("name")

In [27]:
registrations

,reg_id
name,
Andrew,1
Bobo,2
Claire,3
David,4


In [28]:
pd.merge(registrations,logins,left_index=True,right_on='name')

,reg_id,log_id,name
1,1,2,Andrew
3,2,4,Bobo


In [29]:
pd.merge(logins,registrations,right_index=True,left_on='name')

,log_id,name,reg_id
1,2,Andrew,1
3,4,Bobo,2


### Что делать в случае, если ключевые колонки в двух таблицах называются по-разному

In [30]:
registrations = registrations.reset_index()

In [31]:
registrations

,name,reg_id
0,Andrew,1
1,Bobo,2
2,Claire,3
3,David,4


In [32]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [33]:
registrations.columns = ['reg_name','reg_id']

In [34]:
registrations

,reg_name,reg_id
0,Andrew,1
1,Bobo,2
2,Claire,3
3,David,4


In [35]:
pd.merge(registrations,logins,left_on='reg_name',right_on='name')

,reg_name,reg_id,log_id,name
0,Andrew,1,2,Andrew
1,Bobo,2,4,Bobo


In [36]:
pd.merge(registrations,logins,left_on='reg_name',right_on='name').drop('reg_name',axis=1)

,reg_id,log_id,name
0,1,2,Andrew
1,2,4,Bobo


### Pandas автоматически добавляет суффиксы для дублирующихся названий колонок

In [37]:
registrations.columns = ['name','id']

In [38]:
logins.columns = ['id','name']

In [39]:
registrations

,name,id
0,Andrew,1
1,Bobo,2
2,Claire,3
3,David,4


In [40]:
logins

,id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [41]:
# _x для левой таблицы
# _y для правой таблицы
pd.merge(registrations,logins,on='name')

,name,id_x,id_y
0,Andrew,1,2
1,Bobo,2,4


In [42]:
pd.merge(registrations,logins,on='name',suffixes=('_reg','_log'))

,name,id_reg,id_log
0,Andrew,1,2
1,Bobo,2,4
